# Custom Chatbot

using openAI API and customizing the prompt to make it answer from a knowledge bank

In [10]:
import openai
import pandas as pd
import tiktoken

In [17]:
from openai import OpenAI
client = OpenAI(
    base_url = "https://openai.vocareum.com/v1",
    api_key = "api-key"
)

In [18]:
fashion = pd.read_csv("data/2023_fashion_trends.csv")

fashion.head()

,URL,Trends,Source
0,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Red. Glossy red hues took ...,7 Fashion Trends That Will Take Over 2023 — Sh...
1,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Cargo Pants. Utilitarian w...,7 Fashion Trends That Will Take Over 2023 — Sh...
2,https://www.refinery29.com/en-us/fashion-trend...,"2023 Fashion Trend: Sheer Clothing. ""Bare it a...",7 Fashion Trends That Will Take Over 2023 — Sh...
3,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Denim Reimagined. From dou...,7 Fashion Trends That Will Take Over 2023 — Sh...
4,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Shine For The Daytime. The...,7 Fashion Trends That Will Take Over 2023 — Sh...


In [19]:
fashion = fashion.rename({"Trends": "text"}, axis = 1)

fashion.head()

,URL,text,Source
0,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Red. Glossy red hues took ...,7 Fashion Trends That Will Take Over 2023 — Sh...
1,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Cargo Pants. Utilitarian w...,7 Fashion Trends That Will Take Over 2023 — Sh...
2,https://www.refinery29.com/en-us/fashion-trend...,"2023 Fashion Trend: Sheer Clothing. ""Bare it a...",7 Fashion Trends That Will Take Over 2023 — Sh...
3,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Denim Reimagined. From dou...,7 Fashion Trends That Will Take Over 2023 — Sh...
4,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Shine For The Daytime. The...,7 Fashion Trends That Will Take Over 2023 — Sh...


#Custom Query Completion
TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI Completion model. You may copy and paste any useful code from the course materials.

In [20]:
def get_answer(prompt):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": prompt}
    ]

  )

  answer = response.choices[0].message.content
  return answer

In [21]:
tokenizer = tiktoken.get_encoding("cl100k_base")
token_limit = 1000

def get_prompt(question):
  prompt_template = """
  Answer the question based on the context below, and if the
  question can't be answered based on the context, say
  "I don't know"

  Context:

  {}

  ---

  Question: {}
  Answer:"""
  token_count = len(tokenizer.encode(prompt_template)) + \
                          len(tokenizer.encode(question))

  # Create a list to store text for context
  context_list = []

  # Loop over rows of the sorted dataframe
  for text in fashion["text"].values:
      # Append text to context_list if there is enough room
      token_count += len(tokenizer.encode(text))
      if token_count <= token_limit:
          context_list.append(text)
      else:
          # Break once we're over the token limit
          break

  prompt = prompt_template.format(
    "\n\n###\n\n".join(context_list),
    question
    )

  return prompt

#Custom Performance Demonstration
TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic Completion model query as well as the answer from your custom query.

# Question 1

## Answer without context:

In [30]:
answer = get_answer("what are fashion color trends in 2023?")
print(answer)

In 2023, fashion color trends reflected a mix of bold, vibrant hues and softer, more muted tones, catering to a wide range of personal styles and preferences. Here are some notable color trends from 2023:

1. **Vibrant and Bold Colors**: Bright, saturated colors like electric blue, vivid red, and bright orange were prominent, bringing energy and optimism to wardrobes.

2. **Pastels**: Soft pastels continued to be popular, with colors like mint green, lavender, and baby blue providing a gentle, refreshing palette that is both calming and stylish.

3. **Earthy Tones**: Rich, earthy colors such as terracotta, ochre, and moss green gained traction, reflecting a growing interest in sustainability and nature-inspired aesthetics.

4. **Neutrals**: Classic neutral shades, including beige, cream, and taupe, remained essential, offering versatility and a timeless appeal while serving as a backdrop for bolder accessories and pieces.

5. **Deep Jewel Tones**: Colors like emerald green, sapphire bl

## Answer with context:

In [31]:
prompt = get_prompt("what are fashion color trends in 2023?")
answer = get_answer(prompt)
print(answer)

The fashion color trends in 2023 include glossy red hues, cobalt blue, and vibrant colors paired with Pantone's Viva Magenta. Red is showcased in head-to-toe looks and accent accessory pieces, while cobalt blue is noted for its freshness and versatility in daily wear.


There is an obvious in the answer, we can also ask it a question out of context and see what it going to say, we expect it to say I don't know

# Question 2

## Answer without context:

In [27]:
answer = get_answer("what are color trends in 2024?")
print(answer)

While I don't have the ability to predict the future, I can give you insights based on recent trends and emerging patterns in design and fashion that have been observed leading into 2024. Color trends often reflect broader social, cultural, and technological influences.

Here are some anticipated color trends for 2024:

1. **Earth Tones**: There has been a continued interest in earthy colors that evoke a sense of nature and sustainability. Hues like terracotta, moss green, and warm browns are expected to remain popular.

2. **Vibrant and Bold Colors**: Colors that are bright and energetic may make a comeback, especially as people seek to express individuality and optimism. Think bold reds, bright blues, and vivid yellows.

3. **Soft Pastels**: Softer shades like blush pink, lavender, and mint green are likely to continue being popular, providing a calm and soothing aesthetic.

4. **Neutrals with a Twist**: Classic neutrals like beige, gray, and white remain in style but may be mixed wi

# Answer with context:

In [28]:
prompt = get_prompt("what are the fashion trends in 2024?")
answer = get_answer(prompt)
print(answer)

I don't know.


It answers correctly with i don't know as the data doesn't have this information, this means that it was correctly according to the context.